# Byte-Pair Encoding

Dans cette séance, on se propose de réécrire l'algorithme *Byte-Pair encoding* pour améliorer les performances de génération de texte. Nous nous sommes librement inspiré de la [vidéo Youtube](https://www.youtube.com/watch?v=zduSFxRajkE) d'[Andrej Karpathy](https://fr.wikipedia.org/wiki/Andrej_Karpathy) pour réécrire l'algorithme. Puis nous mesurerons le gain de ce travail dans un second temps en utilisant des réseaux de neurones récurrents pour générer du texte.

Commençons par définir une phrase qui nous servira de test pour l'ensemble de la phase de construction de l'algorithme. Elle est issue du roman [Madame Bovary](https://fr.wikipedia.org/wiki/Madame_Bovary) de [Gustave Flaubert](https://fr.wikipedia.org/wiki/Gustave_Flaubert).


In [ ]:
string = """La conversation de Charles était plate comme un trottoir de rue, et les idées de tout le monde y défilaient dans leur costume ordinaire, sans exciter d'émotion, de rire ou de rêverie"""
print(string)

La manière naïve que l'on peut utiliser pour *découper* un texte est de le séparer caractère par caractère puis d'entraîner un algorithme à prédire le prochain caractère. Nous réaliserons cette méthode, et nous allons la comparer à l'approche tokens.

Chaque caractère peut être transformé en un entier à l'aide de la fonction `ord`. La fonction `chr` renvoie le caractère associé à un entier. 

**Consigne** : Tester la fonction `ord` et la fonction `chr` sur les premiers caractères de la chaîne `string`.


In [ ]:
caracters_as_integers = [ord(character) for character in string[:5]]
transcription = [chr(integer) for integer in caracters_as_integers]
print(caracters_as_integers)
print(transcription)

**Question** : Le caractère *L* est-il encodé de la même manière que le caractère *l* ?



La méthode `encode` permet de transformer une chaîne de caractères en une liste d'entiers. Ces entiers sont les mêmes que ceux que l'on a obtenus avec la fonction `ord` si on utilise l'encodage *utf-8*. Nous utiliserons ici l'encodage *latin* parce que nous avons des caractères *spéciaux* comme *é* ou *è*.

**Consigne** : Vérifier l'affirmation précédente, puis encoder la totalité de la chaîne `string` dans la variable `tokens`. La méthode `encode` renvoie une chaîne binaire. Il conviendra de la transformer en une liste d'entier.


Avant de continuer, nous souhaitons avoir une fonction qui va décoder une liste de token en texte lisible.

**Consigne**: Définir la fonction `tokens_to_text` qui prend en paramètre une liste de token et un dictionnaire qui correspond au vocabulaire.

L'algorithme *Byte-Pair encoding* est une méthode de *tokenization* qui consiste à découper un texte en un ensemble de *tokens*. Un *token* est un mot ou une séquence de mots qui sont formés comme un couple de caractères fréquemment adjacent.
Pour être capable d'identifier de tel couple, il faut être capable de calculer pour une liste d'entier la fréquence d'apparition de chaque paire.

**Consigne** : Définir une fonction `get_stats` qui prend en entrée une liste d'entier et qui renvoie un dictionnaire. Le dictionnaire contient les paires d'entiers en tant que clés, et leur nombre d'apparitions en tant que valeurs.

**Consigne** : Tester la fonction `get_stats` sur la liste `tokens` et stocker le résultat dans une variable `stats`.



**Consigne** : Exécuter la cellule-ci dessous pour identifier la paire la plus fréquente, puis traduire les trois premières paires en terme de caractères.

Pour extraire plus simplement la paire la plus fréquente, on peut utiliser la méthode `max` avec la clé `stats.get`.

**Consigne** : Afficher la paire la plus fréquente avec la méthode `max`.

Nous sommes à présent capable d'identifier la paire d'entier la plus fréquente dans une liste d'entier, chacun représentant un *token / caractères*. Nous devons à présent remplacer la paire la plus fréquente par un nouvel index.

**Consigne** : Définir une fonction `merge` qui prend en entrée une liste d'entier, une paire d'entier, et un nouvel index, et qui renvoie une liste d'entier. Cette fonction remplace toutes les occurrences de la paire d'entier par le nouvel index dans la liste d'entier.

**Consigne** : Vérifier le bon fonctionnement de la fonction `merge` avec la liste présente dans la cellule ci-dessous.

L'encoding *utf-8* est un encoding de 32 bits donc il peut représenter 256 différents entiers, numéroté de 0 à 255. Ainsi, nous allons itérativement remplacer dans la chaîne de caractère `string` les paires les plus fréquentes par un nouvel index.

**Consigne** : Définir une fonction `byte_pair_encoding` qui prend en entrée ...


**Consigne** : Utiliser la fonction `byte_pair_encoding` sur la liste `tokens` pour une taille de vocabulaire de 32, puis calculer le pourcentage de réduction de la liste.

## Chargement du texte

Il nous reste à maintenant charger puis processer du texte. Commençons par définir une fonction `clean_text` qui prend en paramètre du texte et qui remplace un ensemble de caractère pour simplifier le travail de tokenization. Par exemple :
* 'à' devient 'a'
* 'é' devient 'e'
* 'æ' devient 'ae'

In [ ]:
def clean_text(text):
    changements = {
        '{': '', '}': '', '°': ' ', 'ç': 'c', 'â': 'a', 'ù': 'u', 'ë': 'e', '<': '', '>': '', '_': '', 'œ': 'oe', '*': '', '^': '', 'á': 'a', 'ô': 'o',
        'ï': 'i', 'æ': 'ae', 'º': '', '«': '"', '»': '"', '’': "'", 'û': 'u', 'ü': 'u', 'ê': 'e', 'à': 'a', 'î': 'i', 'è': 'e', 'é': 'e'
        }
    
    string = text
    for key, value in changements.items():
        string = string.replace(key, value)
    
    return string

Cette fonction s'applique sur du texte: il faut définir une fonction qui va l'extraire. Les fichiers viennent du projet [Gutenberg](https://gutenberg.org/), ils ont donc un format particulier où l'oeuvre est en entier mais ne commence et ne termine pas par l'oeuvre. Il y a du texte spécifiant le livre, les métadonnées et le projet par exemple. Ces informations ne sont pas utile pour notre travail.

**Consigne**: Définir une fonction `load_text` qui prend en paramètre :
* *filename*: nom du fichier de texte
* *sentence_start*: ligne qui indique le début du texte
* *sentence_end*: ligne qui indique la fin du texte
* *train_ratio*: proportition du ratio du texte qui sera dans le dataset d'entraînement et la proporition du texte dans le dataset de test

Cette fonction devra lire le fichier de texte, et extraire l'ensemble du texte entre les deux lignes spécifiées en paramètre. Elle renvoie le texte complet nettoyé avec la fonction `clean_text` dans deux datasets: un d'entraînement et un de test.


On se propose de charger des oeuvres puis d'encoder le texte.

In [ ]:
train_bovary, test_bovary = load_text(
    filename="MadameBovary.txt",
    sentence_start="Nous étions à l'Étude, quand le Proviseur entra, suivi d'un",
    sentence_end="Il vient de recevoir la croix d'honneur."
)

train_education, test_education = load_text(
    filename="EducationSentimentale.txt",
    sentence_start="Le 15 septembre 1840, vers six heures du matin, la Ville-de-Montereau",
    sentence_end="ça me vengera!»"
)

train_chartreuse, test_chartreuse = load_text(
    filename="Chartreuse.txt",
    sentence_start="Le 15 mai 1796, le général Bonaparte fit son entrée dans Milan à la",
    sentence_end="grands-ducs de Toscane."
)

train_rougenoir, test_rougenoir = load_text(
    filename="RougeNoir.txt",
    sentence_start="La petite ville de Verrières peut passer pour l'une des plus jolies de",
    sentence_end="embrassant ses enfants."
)

text_train = train_bovary + train_education + train_chartreuse + train_rougenoir
text_test = test_bovary + test_education + test_chartreuse + test_rougenoir


vocabulary = {index: caracter for index, caracter in enumerate(set(text_train))}
encode = {item[1]: item[0] for item in vocabulary.items()}
tokens_train = list(map(encode.get, text_train))

Il faut maintenant passer du texte encodé aux tokens à proprement parler. On souhaite travailler avec une taille de vocabulaire de 128.

**Consigne**: Tokenizer le texte d'entraînement puis celui de test. Mesurer également la compression réalisée.

A ce stade nous n'avons que du texte mais pas un dataset pour s'entraîner. La fonction `get_dataset` permet de le faire. On décide de travailler avec une longueur de séquence de 64.

In [ ]:
import tensorflow as tf
import numpy as np
import keras


def get_datasets(tokens, sequence_length, stride=1):
    sequences = []
    y_character = []
    for index in range(0, len(tokens) - sequence_length, stride):
        sequences.append(tokens[index:(index + sequence_length)])
        y_character.append(tokens[index + sequence_length])
    
    sequences = keras.utils.pad_sequences(sequences, maxlen=sequence_length)
    X = np.array(sequences)
    y = keras.utils.to_categorical(y_character, num_classes=max_vocab_size)
    return X, y


sequence_length = 64
X_train, y_train = get_datasets(tokens_train, sequence_length)
X_test, y_test = get_datasets(tokens_test, sequence_length)

## Modélisation

Nous avons deux datasets correspondant à du texte tokenize. Il faut maintenant construire le modèle de langage. Pour le faire, on utilisera la couche [Embedding](https://keras.io/api/layers/core_layers/embedding/) en début de modèle pour réduire la dimension du texte, puis on réalise un réseau de neurone récurrent *classique*.

**Consigne**: Définir ce modèle.


**Consigne**: Entraîner le modèle après avoir défini des hyper-paramètres cohérent.